In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from requests.utils import requote_uri
from fake_useragent import UserAgent
from lxml import html
import random
import time
from selenium import webdriver


In [ ]:
FILE = 'C:\\Users\\nusret\\Desktop\\KU Leuven Documents\\Thesis\\inno5_address_toStatVla_cis19.xlsx'
CIS_Survey_Dataframe = pd.read_excel(FILE, sheet_name='inno5_address_toStatVla_cis19')
CIS_Survey_Dataframe.head()

In [49]:
CIS_Survey_Dataframe['Clean_Web'] = ''
for index, row in CIS_Survey_Dataframe.iterrows():
    if ((not pd.isnull(row['cur_web'])) and (not pd.isnull(row['web']))):
        CIS_Survey_Dataframe.loc[index,'Clean_Web'] = 'Needs Checking'
    elif (pd.isnull(row['cur_web']) and (not pd.isnull(row['web']))):
        CIS_Survey_Dataframe.loc[index,'Clean_Web'] = row['web']
    elif ((not pd.isnull(row['cur_web'])) and (pd.isnull(row['web']))):
        CIS_Survey_Dataframe.loc[index,'Clean_Web'] = row['cur_web']
    else: 
        CIS_Survey_Dataframe.loc[index,'Clean_Web'] = 'No Data'
CIS_Survey_Dataframe['Clean_Web']

0                www.farys.be
1                www.pidpa.be
2           www.water-link.be
3                 www.iwva.be
4                 www.imog.be
                ...          
3174               www.tec.be
3175    www.stiebel-eltron.be
3176                  No Data
3177             www.sqli.com
3178                  No Data
Name: Clean_Web, Length: 3179, dtype: object

In [50]:
Total_Businesses = len(CIS_Survey_Dataframe)
No_Website_Count = len(CIS_Survey_Dataframe[CIS_Survey_Dataframe['Clean_Web'] == 'No Data']) 
Dual_Website_Count = len(CIS_Survey_Dataframe[CIS_Survey_Dataframe['Clean_Web'] == 'Needs Checking']) 

print('There are %d Flemish companies in the dataset, %d of them are missing website information. (App. %.02f' 
      %(Total_Businesses,No_Website_Count, (100*(No_Website_Count/Total_Businesses))) + '%)')
print('\nThere are %d Flemish companies that have dual website information in the dataset. (App. %.02f' 
      %(Dual_Website_Count, (100*(Dual_Website_Count/Total_Businesses))) + '%)')

There are 3179 Flemish companies in the dataset, 807 of them are missing website information. (App. 25.39%)

There are 159 Flemish companies that have dual website information in the dataset. (App. 5.00%)


In [ ]:
URL_Check_Daraframe = CIS_Survey_Dataframe.iloc[0:0]
for index, row in CIS_Survey_Dataframe.iterrows():
    if((row['Clean_Web'] == 'No Data') | (row['Clean_Web'] == 'Needs Checking')):
        URL_Check_Daraframe = URL_Check_Daraframe.append(row)
URL_Check_Daraframe = URL_Check_Daraframe.reset_index()
URL_Check_Daraframe

In [ ]:
for index, row in URL_Check_Daraframe.iterrows():
    URL_Check_Daraframe.loc[index, 'Search_Phrase'] = (re.sub('\n', '', re.sub(' +', ' ', str(row['cur_firm']))))
URL_Check_Daraframe_Compact = URL_Check_Daraframe.loc[:,['btw', 'Search_Phrase', 'cur_street1', 'finalzip']]
URL_Check_Daraframe_Compact['Web_Result_1'] = ''
URL_Check_Daraframe_Compact['Web_Result_2'] = ''
URL_Check_Daraframe_Compact['Web_Result_3'] = ''
URL_Check_Daraframe_Compact

In [43]:
Counter = 239
start = time.time()

browser = webdriver.Chrome('C:\\Users\\nusret\\Desktop\\Scrapping\\Drivers\\chromedriver')
browser.get('https://www.google.be/search?q=start')
time.sleep(30)

for index, row in URL_Check_Daraframe_Compact[239:].iterrows():

    Base_URL = requote_uri('https://www.google.be/search?q=' + row['Search_Phrase'] + '&gws_rd=cr')
    
    Random_Number = random.randint(1,10)
    if ('google' in browser.current_url) & (Random_Number < 5):
        input = browser.find_element_by_name('q')
        input.clear()
        time.sleep(random.randint(1,3))
        input.send_keys(row['Search_Phrase'])
        time.sleep(random.randint(1,3))
        try:
            browser.find_element_by_css_selector("#tsf > div:nth-child(2) > div.A8SBwf.emcav > div.RNNXgb > button").click()
        except:
            browser.get(Base_URL)
    elif (Random_Number < 9):
        browser.get(Base_URL)
        time.sleep(random.randint(1,3))
    else:
        Base_URL = requote_uri('https://www.bing.com/search?cc=be&q=' + row['Search_Phrase'])
        browser.get(Base_URL)
        time.sleep(random.randint(1,3))
    
    soup = BeautifulSoup(browser.page_source)
    URL_Check_Daraframe_Compact.loc[index, 'Web_Result_1'] = soup.findAll("cite")[0].text.split('›')[0].rstrip()
    URL_Check_Daraframe_Compact.loc[index, 'Web_Result_2'] = soup.findAll("cite")[1].text.split('›')[0].rstrip()
    URL_Check_Daraframe_Compact.loc[index, 'Web_Result_3'] = soup.findAll("cite")[2].text.split('›')[0].rstrip()
   
    if ((Counter % 10) == 0):
        end = time.time()
        print('Iteration Number ' + str(Counter) + ' Completed - Total Elapsed Time: %.02f minutes' % ((end - start)/60))
    Counter += 1
    time.sleep(random.randint(30,35))

#browser.execute_script("window.scrollBy(0,"+str(random.randint(0,100))+");")


Iteration Number 240 Completed - Total Elapsed Time: 1.28 minutes
Iteration Number 250 Completed - Total Elapsed Time: 7.48 minutes
Iteration Number 260 Completed - Total Elapsed Time: 13.42 minutes
Iteration Number 270 Completed - Total Elapsed Time: 19.25 minutes
Iteration Number 280 Completed - Total Elapsed Time: 25.40 minutes
Iteration Number 290 Completed - Total Elapsed Time: 31.54 minutes
Iteration Number 300 Completed - Total Elapsed Time: 37.45 minutes
Iteration Number 310 Completed - Total Elapsed Time: 43.59 minutes
Iteration Number 320 Completed - Total Elapsed Time: 49.70 minutes
Iteration Number 330 Completed - Total Elapsed Time: 55.64 minutes
Iteration Number 340 Completed - Total Elapsed Time: 61.75 minutes
Iteration Number 350 Completed - Total Elapsed Time: 67.93 minutes
Iteration Number 360 Completed - Total Elapsed Time: 74.20 minutes
Iteration Number 370 Completed - Total Elapsed Time: 80.29 minutes
Iteration Number 380 Completed - Total Elapsed Time: 86.40 minut

In [44]:
URL_Check_Daraframe_Compact.to_pickle("C:\\Users\\nusret\\Desktop\\KU Leuven Documents\\Thesis\\Scraped_URLs.pkl")

In [ ]:
URL_Check_Daraframe_Compact = pd.read_pickle("C:\\Users\\nusret\\Desktop\\KU Leuven Documents\\Thesis\\Scraped_URLs.pkl")
URL_Check_Daraframe_Compact.tail()

In [64]:
Final_Merged_DataFrame = URL_Check_Daraframe.merge(URL_Check_Daraframe_Compact.loc[:,['btw', 'Web_Result_1', 'Web_Result_2', 'Web_Result_3']], on ='btw')
Final_Merged_DataFrame.to_excel("C:\\Users\\nusret\\Desktop\\KU Leuven Documents\\Thesis\\Scraped_URLs.xlsx", sheet_name='URL') 